<a href="https://colab.research.google.com/github/anandhc6/Assignment-3/blob/main/Finetune_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finetuning GPT2 using HuggingFace and Tensorflow**


Setup by cloning transformers repo

In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 93572, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 93572 (delta 93), reused 101 (delta 58), pack-reused 93429
Receiving objects: 100% (93572/93572), 85.83 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (68602/68602), done.


In [2]:
# Clone should now be in the machine
!ls

sample_data  transformers


In [3]:
!nvidia-smi

Tue May 10 06:13:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Changing directory location.


In [4]:
import os
os.chdir("/content/transformers")
os.chdir("./examples/tensorflow/language-modeling")
!ls

README.md  requirements.txt  run_clm.py  run_mlm.py


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 325 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 93.3 MB/s 
     |████████████████████████████████| 1.1 MB 75.7 MB/s 
     |████████████████████████████████| 136 kB 75.1 MB/s 
     |████████████████████████████████| 212 kB 84.5 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
     |████████████████████████████████| 127 kB 81.1 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 98.6 MB/s 
     |████████████████████████████████| 144 kB 88.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.

In [6]:
!ls

README.md  requirements.txt  run_clm.py  run_mlm.py


In [7]:
!pip install pyarrow --upgrade

     |████████████████████████████████| 29.3 MB 1.2 MB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [8]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./tensorflow/language-modeling")

In [9]:
#Installing latest transformer packages
! pip install git+https://github.com/huggingface/transformers/

  Cloning https://github.com/huggingface/transformers/ to /tmp/pip-req-build-03bevd1u
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-03bevd1u
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 88.2 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4115037 sha256=270d2306c6458abce7c7d649b6028ed20aba31bc78a608b6aa76026a4f97afc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ke_4c17t/wheels/fb/1b/91/0fcf504c386d427d65bbaf663eadf8e18cbf9795394ed7050d
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
#Load Data
from sklearn.model_selection import train_test_split

with open('/content/total_data (1).txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
with open('/content/training.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('/content/evaluation.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

The script below will fine tune GPT2 on your text data that you setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using colab or colab pro. We utilize mixed precision in this model to shave off some training time. For a large data set I was using for another experiment it saved us over 30 mins in training time.

In [13]:
!python run_clm.py \
--model_name_or_path distilgpt2 \
--train_file "/content/training.txt" \
--do_train \
--validation_file "/content/evaluation.txt" \
--do_eval \
--per_gpu_train_batch_size 128 \
--save_steps -1 \
--num_train_epochs 15 \
--fp16 \
--output_dir="/content/output_final_1"
#--model_type gpt2-medium \

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/util/structure.py", line 22, in <module>
    from tensorflow.python.data.util import nest
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/util/nest.py", line 36, in <module>
    from tensorflow.python.framework import sparse_tensor as _sparse_tensor
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/sparse_tensor.py", line 24, in <module>
    from tensorflow.python.framework import constant_op
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/constant_op.py", line 25, in <module>
    from tensorflow.python.eager import execute
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager

# **Using the model**
Next lets take our model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("/content/output_final_1", from_pt=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/output_final_1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Encoding sample text is now extremely simple using the pretrained tokenizer.

In [ ]:
input_ids = tokenizer.encode("I love Deep Learning", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([   40,  1842, 10766, 18252], dtype=int32)>

Next we will use the model to generate the text from our input sample. The parameters I used are based on trail and error from playing around with the huggingface tutorial, https://huggingface.co/blog/how-to-generate, which really goes into great detail on how to go about finding the best parameters for generating text. As well they dive into really good information on what each parameter does and how they play into one another.

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=500,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)
beam_output = generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: I love Deep Learning, can't you see?

1: I love Deep Learning, but I need to talk again

2: I love Deep Learning

3: I love Deep Learning

4: I love Deep Learning



In [ ]:
#Print output for each sequence generated above
beam_output = generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

0: I love Deep Learning, can't you see?

1: I love Deep Learning, but I need to talk again

2: I love Deep Learning

3: I love Deep Learning

4: I love Deep Learning

